This notebook assumes that you have a running instance of Neo4J Aura. You can create a free one by heading over to https://neo4j.com/

Once you have your instance, save its id as `neo4j_instance_id` and its password as `neo4j_password` as Colab secrets and grant this notebook access to said secrets.

In [1]:
!pip install neo4j yfiles_jupyter_graphs_for_neo4j pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.0 MB/s eta 0:00:00


In [2]:
from neo4j import GraphDatabase

In [3]:
from google.colab import userdata
neo4j_instance_id = userdata.get('neo4j_instance_id')
neo4j_password = userdata.get('neo4j_password')

driver = GraphDatabase.driver(f"neo4j+s://{neo4j_instance_id}.databases.neo4j.io:7687", auth=("neo4j", neo4j_password))

In [4]:
def insert_data(tx):
    query = """
    CREATE (w:Person {name: 'Walt Whitman'})
    CREATE (p:Person {name: 'Pablo Neruda'})
    CREATE (n:Person {name: 'Nasir Kazmi'})
    CREATE (s:Person {name: 'Sylvia Plath'})
    CREATE (c:Person {name: 'Charles Bukowski'})

    CREATE (w)-[:FRIEND]->(p)
    CREATE (w)-[:FRIEND]->(n)
    CREATE (p)-[:FRIEND]->(n)
    CREATE (n)-[:FRIEND]->(s)
    CREATE (s)-[:FRIEND]->(c)

    RETURN *
    """
    tx.run(query)

with driver.session() as session:
    session.execute_write(insert_data)

In [10]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget

g = Neo4jGraphWidget(driver)

def show_graph(driver):
    query = """
    MATCH (n)-[r]->(m)
    RETURN n, r, m
    """
    g.show_cypher(query)

show_graph(driver)

GraphWidget(layout=Layout(height='500px', width='100%'))

In [9]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [7]:
from google.colab import output
output.disable_custom_widget_manager()

In [11]:
import pandas as pd

def show_degree_centrality(driver):
    query = """
      MATCH (p:Person)
      RETURN p.name AS Person,
            size([(p)--() | 1]) AS DegreeCentrality
      ORDER BY DegreeCentrality DESC
    """
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

results = show_degree_centrality(driver)
pd.DataFrame(results)

,Person,DegreeCentrality
0,Nasir Kazmi,3
1,Walt Whitman,2
2,Pablo Neruda,2
3,Sylvia Plath,2
4,Charles Bukowski,1
